<a href="https://colab.research.google.com/github/ShawnTayNWU/MSDS460/blob/Assignments/MSDS_460_Q3_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MSDS 460 Assignment 4 Question 3
This is code generated to answer assignment 4 question 3

Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Install and Import Packages

In [2]:
!pip install simpy

In [3]:
import itertools
import random


import random

import numpy as np # for random number distributions
import pandas as pd # for event_log data frame
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 10)
import simpy # discrete event simulation environment
from sympy.utilities.iterables import multiset_permutations  # generator

Set up Inputs

In [ ]:
RANDOM_SEED =  50
MATERIAL_LEVEL = [5, 25]   # assumed materials min and max
Worker_Speed = 2           # worker speed per hour
T_INTER = [30, 301]        # start a work every [min, max] seconds        
SIM_TIME = 100350
simulation_runs = 10       # the number of home-building projects


total_workers_by_activity_Q3 = [ 1,  1,  1, 1,  1,   1,   1, 1]  # number of workers needed
total_workers_by_activity_Q4a = [ 1,  1,  1, 1,  1,   2,   2, 1]  # number of workers needed
total_workers_by_activity_Q4b = [ 1,  3,  4, 1,  1,   2,   2, 1]  # number of workers needed

list_workers = [total_workers_by_activity_Q3,total_workers_by_activity_Q4a, total_workers_by_activity_Q4b ]

Set up simulation paramters

In [ ]:
def calculate_mode_by_trace(trace): 
    
    mode = []
    for x in range(len(trace)):
        if trace[x] == 'dig':
            mode.append(4)
        elif trace[x] == 'bricks':
            mode.append(7)
        elif trace[x] == 'tiles':
            mode.append(12)
        elif trace[x] == 'lay':
            mode.append(2)
        elif trace[x] == 'walls':
            mode.append(10)
        elif trace[x] == 'wiring':
            mode.append(5)
        elif trace[x] == 'roofing':
            mode.append(3)
        else:
            mode.append(4)
            
    return mode
#[2, 3.5,  6, 1,  5, 2.5, 1.5, 2]
def calculate_minima_by_trace(trace): 
    
    minima = []
    for x in range(len(trace)):
        if trace[x] == 'dig':
            minima.append(2)
        elif trace[x] == 'bricks':
            minima.append(3.5)
        elif trace[x] == 'tiles':
            minima.append(6)
        elif trace[x] == 'lay':
            minima.append(1)
        elif trace[x] == 'walls':
            minima.append(5)
        elif trace[x] == 'wiring':
            minima.append(2.5)
        elif trace[x] == 'roofing':
            minima.append(1.5)
        else:
            minima.append(2)
            
    return minima
#[8,  14, 24, 4, 20,  10,   6, 8]
def calculate_maxima_by_trace(trace): 
    
    maxima = []
    for x in range(len(trace)):
        if trace[x] == 'dig':
            maxima.append(8)
        elif trace[x] == 'bricks':
            maxima.append(14)
        elif trace[x] == 'tiles':
            maxima.append(24)
        elif trace[x] == 'lay':
            maxima.append(4)
        elif trace[x] == 'walls':
            maxima.append(20)
        elif trace[x] == 'wiring':
            maxima.append(10)
        elif trace[x] == 'roofing':
            maxima.append(6)
        else:
            maxima.append(8)
            
    return maxima



def print_end(num_worker, time_spent, name, end_time):
    if num_worker == 1:
        print('\t\t%s worker ends work at %.1f seconds.' % (num_worker, time_spent))
    else:
        print('\t\t%s workers ends work at %.1f seconds.' % (num_worker, time_spent))
        
    print('\tEnd %s activity at %.1f end time and took  %.1f seconds.\n' % (name, end_time,
                                                      time_spent))
    
    
    
def print_start(num_worker, start):
    if num_worker == 1:
        print('\t\t%s worker starts works at %.1f seconds.' % (num_worker, start))
    else:
        print('\t\t%s workers starts works at %.1f seconds.' % (num_worker, start))
        
        

def service_process(name, env, station, row, index, trace_list):

    random.randint(*MATERIAL_LEVEL)
    #print('\tStart %s activity at %.1f.' % (name, env.now))
    with station.request() as req:
        start = env.now
        #print_start(row['nworkers'], start)

        yield req

        env.timeout(1)
        
        left  = row['minimum']
        mode  = row['mode']
        right = row['maximum']
        
        this_service_time = np.random.triangular(left, mode, right, size=1)
        yield env.timeout(this_service_time / Worker_Speed)
        
        time_spent = env.now - start
        #Add to dataframe
        trace_list.append([index,name, round(time_spent[0],2)])
        #print_end(row['nworkers'],time_spent, name, env.now )

def start(env, station, df, trace_list):
    
    x = 0
    while True: 
        
     yield env.timeout(0)
     
     x +=1
     #print('Home Construction Simulation ' + str(x))
     
     for index, row in df.iterrows():
        yield env.timeout(random.randint(*T_INTER))
        env.process(service_process(row['activity'], env, station, row, x, trace_list))
        
        if row['activity'] == 'painting':
           yield env.timeout(random.randint(*T_INTER))
           #print('\n')

Run Simulation

In [ ]:
# Setup and start the simulation
#print('Home Construction Simulation')
random.seed(RANDOM_SEED)
final_list = []



#print(random.randint(*T_INTER))
# Create environment and start processes


# one-word identifiers for home building activities
activities = ['dig','bricks','tiles', 'lay', 'walls', 'wiring', 'roofing', 
'painting']    
all_traces = [] # initialize list of lists
valid_traces = []  # initialize list of lists
for trace in multiset_permutations(activities):
    all_traces.append(trace)
    # identify locations of activities in the trace
    dig_index = trace.index('dig') 
    bricks_index = trace.index('bricks') 
    tiles_index = trace.index('tiles') 
    lay_index = trace.index('lay')
    walls_index = trace.index('walls') 
    wiring_index = trace.index('wiring')
    roofing_index = trace.index('roofing')
    painting_index = trace.index('painting')
    if (dig_index == 0 or bricks_index == 0 or tiles_index == 0)\
    and (dig_index < lay_index)\
    and (lay_index < walls_index)\
    and (bricks_index < walls_index)\
    and (walls_index < wiring_index)\
    and (tiles_index < roofing_index)\
    and (walls_index < roofing_index)\
    and (wiring_index < painting_index)\
    and (roofing_index < painting_index):
        valid_traces.append(trace)


for worker in list_workers:
    
    for this_trace in valid_traces:
    
        minima = calculate_minima_by_trace(this_trace)
        modes  = calculate_mode_by_trace(this_trace)
        maxima = calculate_maxima_by_trace(this_trace)
    
    
        df_info = pd.DataFrame({'activity': this_trace,
                                 'minimum': minima,
                                 'mode': modes,
                                 'maximum': maxima,
                                 'nworkers': worker})
        overall_list = []
    
    

        column_names = ["Simulation Index", "activity", "runtime"]
        #print(this_trace)
        env = simpy.Environment()
        activity_station = simpy.Resource(env, 8)
        env.process(start(env, activity_station,df_info, overall_list))

        # Execute!
        env.run(until=SIM_TIME)

        df = pd.DataFrame(overall_list)
        df.columns = column_names

        df2 = df.groupby(
            ['Simulation Index'], as_index=False).agg({'runtime':sum})

        df2 = df2.rename(columns={"runtime": "sum"})
        overall_mean = df2["sum"].mean()
        trace_row = [this_trace,worker,minima,modes,maxima,  65, overall_mean]

        final_list.append(trace_row)

Generate Outputs

In [4]:
    #print(final_list)
    #print('\n')


    df_final = pd.DataFrame(final_list)
    df_final.columns = ["Trace Activity", 'Total Number of workers By Trace Activity','Minima','Modes','Maxima',"Overall Runs","Mean Simulation Run"]

    print(df_final)
    if worker == [ 1,  1,  1, 1,  1,   1,   1, 1]:
        df_final.to_csv('Assignment4_Q3.csv', index=False)
    elif worker ==  [ 1,  1,  1, 1,  1,   2,   2, 1]:
        df_final.to_csv('Assignment4_Q4a.csv', index=False)    
    elif worker ==  [ 1,  3,  4, 1,  1,   2,   2, 1]:
        df_final.to_csv('Assignment4_Q4b.csv', index=False)

                                       Trace Activity  \
0   [bricks, dig, lay, tiles, walls, roofing, wiri...   
1   [bricks, dig, lay, tiles, walls, wiring, roofi...   
2   [bricks, dig, lay, walls, tiles, roofing, wiri...   
3   [bricks, dig, lay, walls, tiles, wiring, roofi...   
4   [bricks, dig, lay, walls, wiring, tiles, roofi...   
..                                                ...   
28  [tiles, bricks, dig, lay, walls, wiring, roofi...   
29  [tiles, dig, bricks, lay, walls, roofing, wiri...   
30  [tiles, dig, bricks, lay, walls, wiring, roofi...   
31  [tiles, dig, lay, bricks, walls, roofing, wiri...   
32  [tiles, dig, lay, bricks, walls, wiring, roofi...   

   Total Number of workers By Trace Activity                          Minima  \
0                   [1, 1, 1, 1, 1, 1, 1, 1]  [3.5, 2, 1, 6, 5, 1.5, 2.5, 2]   
1                   [1, 1, 1, 1, 1, 1, 1, 1]  [3.5, 2, 1, 6, 5, 2.5, 1.5, 2]   
2                   [1, 1, 1, 1, 1, 1, 1, 1]  [3.5, 2, 1, 5, 6, 1.5, 2.5, 2